In [1]:
import csv
from bs4 import BeautifulSoup
import requests
import os
from tqdm import tqdm
import pandas as pd

In [2]:
from urllib.parse import urlparse

# Функция для получения имени файла из URL
def get_filename_from_url(url):
    parsed_url = urlparse(url)
    return os.path.basename(parsed_url.path)

In [3]:
df = pd.read_csv('flats_unique.csv')
flat_urls = df['link']

In [4]:
dir_images = 'images'
FILE_OUT_NAME = 'url_images.csv'

START_ID = ...
END_ID = ...

csv_errors = open('download_errors.csv', mode='w', newline='', encoding='utf-8')
writer_err = csv.writer(csv_errors)
writer_err.writerow(['flat_url', 'image_url', 'image_name'])


csv_file = open(FILE_OUT_NAME, mode='w', newline='', encoding='utf-8')
writer = csv.writer(csv_file)

writer.writerow(['flat_url', 'image_url', 'image_name'])

31

In [5]:
%%time

for url_id in tqdm(range(START_ID, END_ID+1), desc='Flat images parsing'):
    flat_link = flat_urls[url_id]
    flat_req = requests.get(flat_link)
    flat_soup = BeautifulSoup(flat_req.text, "html.parser")

    # Находим div с классом "fotorama"
    fotorama_div = flat_soup.find('div', class_='fotorama')

    # Ищем все ссылки на фото внутри этого блока
    image_tags = fotorama_div.find_all('a', href=True)

    # Скачиваем все изображения и записываем данные в CSV
    for i, tag in enumerate(image_tags):
        image_url = tag['href']
        response = requests.get(image_url)

        # Проверяем успешность запроса
        if response.status_code == 200:
            # Извлекаем оригинальное имя файла из URL
            filename = get_filename_from_url(image_url)
            filepath = os.path.join('images', filename)

            with open(filepath, 'wb') as f:
                f.write(response.content)

            # Записываем данные в CSV файл
            writer.writerow([flat_link, image_url, filename])
            print(f'Фото {i+1} скачано как {filename} и записано в CSV.')
        else:
            writer_err.writerow([flat_link, image_url, filename])
            print(f'Ошибка при скачивании фото {i+1}.')

    print(f'--------end of flat {url_id} / {END_ID} ---------')


csv_file.close()
csv_errors.close()

print('End of program')

Flat images parsing:   0%|                                                                       | 0/2 [00:00<?, ?it/s]

Фото 1 скачано как 4be33ca5dc3f60a1271981f99c81e753_1200x900.jpg и записано в CSV.
Фото 2 скачано как ed7b958860f43831bf8256814b90097a_1200x900.jpg и записано в CSV.
Фото 3 скачано как ade322ea624ca6ca61d6eb40673da9c1_1200x900.jpg и записано в CSV.
Фото 4 скачано как 16e862604a3fb954c4e8bf8f2a7633ed_1200x900.jpg и записано в CSV.
Фото 5 скачано как 7c18eec3e249b2c8b6e213eab57214c3_1200x900.jpg и записано в CSV.
Фото 6 скачано как 8c476d751d02c63ed6e3526e2d90c911_1200x900.jpg и записано в CSV.
Фото 7 скачано как 2e20d2049dd35dcfd2a6fda72ad5c744_1200x900.jpg и записано в CSV.
Фото 8 скачано как 070ab1244ed51e52f0f4b18aeea13d32_1200x900.jpg и записано в CSV.
Фото 9 скачано как f28466b3c9d47f483144de479aa2cf62_1200x900.jpg и записано в CSV.


Flat images parsing:  50%|███████████████████████████████▌                               | 1/2 [00:17<00:17, 17.07s/it]

Фото 10 скачано как 69c4828c0bfc38de3df569e341a783c5_1200x900.jpg и записано в CSV.
--------end of flat 0 / 2 ---------
Фото 1 скачано как 94ee478ce2762d236cdc457d5b555571_1200x900.jpg и записано в CSV.
Фото 2 скачано как 0ae8bcd8c8492aaef5d227ac288e4a5b_1200x900.jpg и записано в CSV.
Фото 3 скачано как 2a477d175e6e76a3ab563fb74fb74816_1200x900.jpg и записано в CSV.
Фото 4 скачано как 96ac35a42eaa1a9ec3b3c76470043ad8_1200x900.jpg и записано в CSV.
Фото 5 скачано как 46f3e6c59b38cfcaccb997d63914244d_1200x900.jpg и записано в CSV.
Фото 6 скачано как 4c79e639ce626839f93aacf596e15706_1200x900.jpg и записано в CSV.
Фото 7 скачано как 5b49e663602e0668f47f8bc27886b090_1200x900.jpg и записано в CSV.
Фото 8 скачано как f9109d027bcf1a7ca06e29f3b4470653_1200x900.jpg и записано в CSV.
Фото 9 скачано как 119ff22101e741d45d01db127b51b26b_1200x900.jpg и записано в CSV.
Фото 10 скачано как 73ce8762683f05f0029de6d8f06d266b_1200x900.jpg и записано в CSV.


Flat images parsing: 100%|███████████████████████████████████████████████████████████████| 2/2 [00:46<00:00, 23.06s/it]

Фото 11 скачано как 11c721459d3dea20f7be23a2fcc473dd_1200x900.jpg и записано в CSV.
--------end of flat 1 / 2 ---------
End of program
CPU times: total: 10.3 s
Wall time: 46.1 s
